In [1]:
import pandas as pd

link_df = pd.read_csv('/content/drive/MyDrive/data/NCKH_/Code + Data/cleaned_df.csv', index_col=0 )

In [2]:
link_df = link_df.set_axis(['target', 'source'], axis=1, inplace=False)

In [3]:
link_df.head()

,target,source
0,61523852c288cc4da5ee2e81,6152384cc288cc4da5ee2e80
1,61523852c288cc4da5ee2e81,61523858c288cc4da5ee2e82
2,61523852c288cc4da5ee2e81,6152385fc288cc4da5ee2e83
3,616880cbf6dccfb5cf3794f5,616880b7f6dccfb5cf3794f2
4,616880cbf6dccfb5cf3794f5,616880c0f6dccfb5cf3794f3


In [4]:
authors=pd.read_csv('/content/drive/MyDrive/data/NCKH_/Code + Data/final_3_pca.csv',index_col=0)

In [5]:
authors.head()

,pca_0,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9,topic
Id,,,,,,,,,,,
614b3d356b8ee4b71c58ab7d,-0.183264,-0.243059,-0.340974,0.064406,-0.009195,0.043313,0.161301,0.073235,0.092480,-0.318844,5
614b3d436b8ee4b71c58ab7f,0.426354,0.288682,-0.235454,0.137301,-0.332483,0.042643,0.553443,-0.084734,-0.490196,0.135940,3
614b3da26b8ee4b71c58ab86,0.595626,0.264286,0.095240,0.605487,-0.374734,0.137878,-0.348286,-0.028486,-0.118190,-0.209408,5
614b3da86b8ee4b71c58ab87,0.595626,0.264286,0.095240,0.605487,-0.374734,0.137878,-0.348286,-0.028486,-0.118190,-0.209408,5
614b3dbf6b8ee4b71c58ab88,0.595626,0.264286,0.095240,0.605487,-0.374734,0.137878,-0.348286,-0.028486,-0.118190,-0.209408,5


In [6]:
# install StellarGraph if running on Google Colab
import sys
if 'google.colab' in sys.modules:
  %pip install -q stellargraph[demos]==1.2.1

     |████████████████████████████████| 435 kB 14.8 MB/s 
     |████████████████████████████████| 500 kB 63.3 MB/s 
     |████████████████████████████████| 121 kB 64.7 MB/s 
     |████████████████████████████████| 1.6 MB 62.9 MB/s 
     |████████████████████████████████| 83 kB 2.6 MB/s 
     |████████████████████████████████| 41 kB 710 kB/s 


In [7]:
# verify that we're using the correct version of StellarGraph for this notebook
from stellargraph import StellarDiGraph
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.layer import GraphSAGE, HinSAGE, link_classification

from tensorflow import keras
from sklearn import preprocessing, feature_extraction, model_selection

from stellargraph import globalvar
from stellargraph import datasets
from IPython.display import display, HTML
%matplotlib inline

In [8]:
import networkx as nx

G = nx.from_pandas_edgelist(link_df, source='source', target='target', create_using=nx.Graph())

#Add node attributes to the graph

for node in G.nodes():
    try:
        G.nodes[node]['pca'] = authors.loc[node].values.tolist()
    except:
        G.nodes[node]['pca'] = [0]*len(authors.columns)



In [9]:
print(nx.info(G))

Graph with 15895 nodes and 30866 edges


<ipython-input-9-60d20efbd280>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(G))


In [10]:
from stellargraph import StellarGraph

author_one_hot_subject = StellarGraph.from_networkx(G, node_features="pca")

print(author_one_hot_subject.info())

StellarGraph: Undirected multigraph
 Nodes: 15895, Edges: 30866

 Node types:
  default: [15895]
    Features: float32 vector, length 11
    Edge types: default-default->default

 Edge types:
    default-default->default: [30866]
        Weights: all 1 (default)
        Features: none


# Split train test

In [11]:
# Define an edge splitter on the original graph G:
edge_splitter_test = EdgeSplitter(author_one_hot_subject)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from G, and obtain the
# reduced graph G_test with the sampled links removed:
G_test, edge_ids_test, edge_labels_test = edge_splitter_test.train_test_split(
    p=0.1, method="global", keep_connected=True
)

** Sampled 3086 positive and 3086 negative edges. **


In [12]:
# Define an edge splitter on the reduced graph G_test:
edge_splitter_train = EdgeSplitter(G_test)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from G_test, and obtain the
# reduced graph G_train with the sampled links removed:
G_train, edge_ids_train, edge_labels_train = edge_splitter_train.train_test_split(
    p=0.1, method="global", keep_connected=True
)

** Sampled 2778 positive and 2778 negative edges. **


In [13]:
print(G_train.info())


StellarGraph: Undirected multigraph
 Nodes: 15895, Edges: 25002

 Node types:
  default: [15895]
    Features: float32 vector, length 11
    Edge types: default-default->default

 Edge types:
    default-default->default: [25002]
        Weights: all 1 (default)
        Features: none


In [14]:
print(G_test.info())

StellarGraph: Undirected multigraph
 Nodes: 15895, Edges: 27780

 Node types:
  default: [15895]
    Features: float32 vector, length 11
    Edge types: default-default->default

 Edge types:
    default-default->default: [27780]
        Weights: all 1 (default)
        Features: none


In [15]:
batch_size = 20
epochs = 5

In [16]:
num_samples = [20, 10]


In [17]:
train_gen = GraphSAGELinkGenerator(G_train, batch_size, num_samples)
train_flow = train_gen.flow(edge_ids_train, edge_labels_train, shuffle=True)

In [18]:
test_gen = GraphSAGELinkGenerator(G_test, batch_size, num_samples)
test_flow = test_gen.flow(edge_ids_test, edge_labels_test)

In [19]:
layer_sizes = [20, 20]
graphsage = GraphSAGE(
    layer_sizes=layer_sizes, generator=train_gen, bias=True, dropout=0.3
)

In [ ]:
# Build the model and expose input and output sockets of graphsage model
# for link prediction
x_inp, x_out = graphsage.in_out_tensors()

In [ ]:
prediction = link_classification(
    output_dim=1, output_act="relu", edge_embedding_method="ip"
)(x_out)

In [ ]:
model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.binary_crossentropy,
    metrics=["acc"],
)

In [ ]:
model.summary()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.vis_utils import plot_model

plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
init_train_metrics = model.evaluate(train_flow)
init_test_metrics = model.evaluate(test_flow)

print("\nTrain Set Metrics of the initial (untrained) model:")
for name, val in zip(model.metrics_names, init_train_metrics):
    print("\t{}: {:0.4f}".format(name, val))

print("\nTest Set Metrics of the initial (untrained) model:")
for name, val in zip(model.metrics_names, init_test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

In [ ]:
%%time
history = model.fit(train_flow, epochs=epochs, validation_data=test_flow, verbose=2)


In [ ]:
print(history.history.keys())

In [ ]:
# summarize history for accuracy
import matplotlib.pyplot as plt

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.grid()
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.grid()
plt.show()

In [ ]:
train_metrics = model.evaluate(train_flow)
test_metrics = model.evaluate(test_flow)

print("\nTrain Set Metrics of the trained model:")
for name, val in zip(model.metrics_names, train_metrics):
    print("\t{}: {:0.4f}".format(name, val))

print("\nTest Set Metrics of the trained model:")
for name, val in zip(model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

Classification report

In [ ]:
from sklearn.metrics import classification_report
predicted_link = model.predict(test_flow, workers=1, verbose=0)
predicted_link = predicted_link.squeeze()
predicted_link = predicted_link.tolist()

In [ ]:
predicted_link = [1 if x > 0.5 else 0 for x in predicted_link]
print(classification_report(predicted_link,edge_labels_test))

In [ ]:
predicted_link_df = pd.DataFrame({"source":edge_ids_test[:,0], "target":edge_ids_test[:,1], "link":predicted_link})
G_test_nx = G_test.to_networkx()
G_test_nx.add_edges_from(predicted_link_df[predicted_link_df["link"]==1][["source", "target"]].values.tolist())


In [ ]:
print(nx.info(G_test_nx))

In [ ]:
df_authors=pd.read_csv('/content/drive/MyDrive/data/NCKH_/Code + Data/WordMap/Author_Institute_Lat_Long_Topic.csv',index_col=0)

In [ ]:

f = plt.figure(figsize=(15,15))
nx.draw(G_test_nx, node_size=3, node_color=df_authors["topic"].map(lambda x: hash(x) % 20),pos=nx.spring_layout(G_test_nx), ax=f.add_subplot(111))
f.savefig("/content/drive/MyDrive/data/NCKH_/Code + Data/Image/GraphSage_100.png")

# Worlmap Plot

In [ ]:
edge_ids_test=edge_ids_test.tolist()
print(edge_ids_test)

In [ ]:
test_df=pd.DataFrame(edge_ids_test,columns=['source','target'])
test_df['label']=edge_labels_test
test_df=test_df[test_df['label']==1]

#drop the test_df column
test_df=test_df.drop(['label'], axis=1)
#reset the index
test_df=test_df.reset_index(drop=True)

list_test=test_df.values.tolist()
new_list=[]

for i in link_df.values.tolist():
    if i not in list_test:
        new_list.append(i)

new_link_df=pd.DataFrame(new_list,columns=['source','target'])

In [ ]:
# datafame with list_link_remove and predicted_link
predicted_df=pd.DataFrame(edge_ids_test, columns = ['source', 'target'])

predicted_df['predicted_link']=predicted_link

#remove the predicted link 0
predicted_df=predicted_df[predicted_df['predicted_link']==1]

#drop the predicted_link column
predicted_df=predicted_df.drop(['predicted_link'], axis=1)

#reset the index
predicted_df=predicted_df.reset_index(drop=True)

predicted_df.head()

In [ ]:
#edge trace
from tqdm import tqdm
Longitude_source=[]

Latitude_source=[]

Longitude_target=[]

Latitude_target=[]

for i in tqdm(range(len(predicted_df))):
    
    source=predicted_df['source'][i]
    #print(source)
    target=predicted_df['target'][i]
    #print(target)
    Longitude_source.append(df_authors[df_authors['Author']==source]['Longitude'].values[0])
    Latitude_source.append(df_authors[df_authors['Author']==source]['Latitude'].values[0])
    Longitude_target.append(df_authors[df_authors['Author']==target]['Longitude'].values[0])
    Latitude_target.append(df_authors[df_authors['Author']==target]['Latitude'].values[0])
    

In [ ]:
#edge trace following new_link_df
from tqdm import tqdm
Longitude_node1 = []
Latitude_node1 = []

Longitude_node2 = []
Latitude_node2 = []

for i in tqdm(range(len(new_link_df))):
    Longitude_node1.append(df_authors[df_authors['Author']==new_link_df['source'][i]]['Longitude'].values[0])
    Latitude_node1.append(df_authors[df_authors['Author']==new_link_df['source'][i]]['Latitude'].values[0])
    
    Longitude_node2.append(df_authors[df_authors['Author']==new_link_df['target'][i]]['Longitude'].values[0])
    Latitude_node2.append(df_authors[df_authors['Author']==new_link_df['target'][i]]['Latitude'].values[0])
    

In [ ]:
# Visualize the network earth map

import plotly.graph_objects as go
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

#edge trace 

edge_trace = go.Scattergeo(
    lon = Longitude_node1+Longitude_node2,
    lat = Latitude_node1+Latitude_node2,
    mode = 'lines',
    line = dict(width = 0.05, color = 'blue'),
    hoverinfo = 'none',
    opacity = 0.1
)

edge_trace_predicted = go.Scattergeo(
    lon = Longitude_source+Longitude_target,
    lat = Latitude_source+Latitude_target,
    #dash = 'dash',
    mode = 'lines',
    line = dict(width = 0.2, color = 'red'),
    hoverinfo = 'none',
    opacity = 0.1
)

merge_edge_trace = [edge_trace, edge_trace_predicted]

#node trace
node_trace = go.Scattergeo(
    lon = df_authors['Longitude'],
    lat = df_authors['Latitude'],
    mode = 'markers',
    hoverinfo = 'text',
    marker = dict(
        size = 2,
        # map color to topic
        color = df_authors['topic'].map(lambda x: hash(x) % 20),
        line_width = 0.5,
        sizemode = 'area'
    )
)
node_trace.text = df_authors['Institute']

fig = go.Figure(data=[node_trace,edge_trace_predicted, edge_trace],
    layout=go.Layout(
        title_text = 'Co-authorship network of authors',
        showlegend = False,
        geo = dict(

            scope = 'world',
            projection_type = 'equirectangular',
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth = 1,
            countrywidth = 1,
            subunitcolor = "rgb(255, 255, 255)",
            countrycolor = "rgb(255, 255, 255)"
        ),
    )
)

fig.show()
# to html
fig.write_html('Network_Map.html')


In [ ]:
from stellargraph.mapper import GraphSAGELinkGenerator, GraphSAGENodeGenerator
# Build the model to predict node representations from node features with the learned GraphSAGE model parameters
x_inp_src = x_inp[0::2]
x_out_src = x_out[0]
embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)
graph_node_list = list(G_train.nodes())
# Get representations for all nodes in ``graph``
node_gen = GraphSAGENodeGenerator(G_train, batch_size, num_samples).flow(graph_node_list)
node_embeddings = embedding_model.predict(node_gen, workers=1, verbose=0)

In [ ]:
node_embeddings[1]